# Validate MOTBX resources

Validate MOTBX resources (YAML files) against JSON schema using *jsonschema*.
A report is written containing file names of resources that fauled validation.
Validation can fail when resource YAML file does not comply with JSON schema, or when URL is invalid or not live.

In [1]:
import os
import sys
from pathlib import Path
from motbxtools import motbxschema
import pprint
pp = pprint.PrettyPrinter(indent = 2)

CWD = Path.cwd()
if CWD.name != "notebooks":
    print("Make sure to run this notebook from the 'notebooks' directory.")

MOTBX_DIR = CWD.parent # home directory of this repository

# path to directory where resources YAML file are saved
RESOURCES_DIR = MOTBX_DIR.joinpath("resources/curated")

# path to JSON SCHEMA file defining structure of MOTBX resources
SCHEMA_JSON = MOTBX_DIR.joinpath("schema/motbxschema.json")

# path to validation report
# - file that contains info about failed validations of MOTBX resources
VALIDATION_REPORT = MOTBX_DIR.joinpath(
    "resources/summary/validation_report.txt")

In [2]:
# load MOTBX schema used for validation
schema = motbxschema.MotbxSchema(SCHEMA_JSON)

with open(VALIDATION_REPORT, "w") as fp:
    print("VALIDATION REPORT - MOTBX resources that failed validation", file=fp)
    print(80*"-", file=fp)
    for root, dirs, files in os.walk(RESOURCES_DIR):
        for name in files:
            if not name.endswith(".yaml"):
                continue
            print(name, end="\r")
            # load MOTBX resource
            resource = motbxschema.MotbxResource(os.path.join(root, name))
            try:
                # validate MOTBX resource
                resource.validate(schema)
            except Exception as error:
                # print validation errors to validation report file
                print(error, file=fp)
                print("Resource:", name, file=fp)
                print("URL:", resource.resource["resourceUrl"], file=fp)
                print(80*"-", file=fp)